In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def load_data():
    """
    This function loads the data from the csv files and returns it as numpy arrays.

    input: None
    
    output: x_pretrain: np.ndarray, the features of the pretraining set
            y_pretrain: np.ndarray, the labels of the pretraining set
            x_train: np.ndarray, the features of the training set
            y_train: np.ndarray, the labels of the training set
            x_test: np.ndarray, the features of the test set
    """
    x_pretrain = pd.read_csv("./pretrain_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_pretrain = pd.read_csv("./pretrain_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_train = pd.read_csv("./train_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_train = pd.read_csv("./train_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_test = pd.read_csv("./test_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1)
    scaler = StandardScaler()
    x_pretrain = scaler.fit_transform(x_pretrain)
    x_train = scaler.transform(x_train)
    x_test_transed = scaler.transform(x_test)
    x_test[x_test.columns] = x_test_transed

    return x_pretrain, y_pretrain, x_train, y_train, x_test

In [4]:
class Net(nn.Module):
    """
    The model class, which defines our feature extractor used in pretraining.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        # TODO: Define the architecture of the model. It should be able to be trained on pretraing data 
        # and then used to extract features from the training and test data.
        self.fc1 = nn.Linear(1000, 1000)
        self.fc2 = nn.Linear(1000, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 64)
        self.fc5 = nn.Linear(64, 1)
        
        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)
        nn.init.xavier_normal_(self.fc3.weight)
        nn.init.xavier_normal_(self.fc4.weight)
        nn.init.xavier_normal_(self.fc5.weight)

    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        # TODO: Implement the forward pass of the model, in accordance with the architecture 
        # defined in the constructor.
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x
    
    def make_feature(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        return x

In [5]:
def make_feature_extractor(x, y, batch_size=256, eval_size=1000):
    """
    This function trains the feature extractor on the pretraining data and returns a function which
    can be used to extract features from the training and test data.

    input: x: np.ndarray, the features of the pretraining set
              y: np.ndarray, the labels of the pretraining set
                batch_size: int, the batch size used for training
                eval_size: int, the size of the validation set
            
    output: make_features: function, a function which can be used to extract features from the training and test data
    """
    # Pretraining data loading
    in_features = x.shape[-1]
    x_tr, x_val, y_tr, y_val = train_test_split(x, y, test_size=eval_size, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)
    train_dataset = torch.utils.data.TensorDataset(x_tr, y_tr)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    # model declaration
    model = Net()
    model.to(device)
    model.train()
    
    # TODO: Implement the training loop. The model should be trained on the pretraining data. Use validation set 
    # to monitor the loss.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = 100
    for epoch in tqdm(range(epochs)):
        loss_tr = 0
        loss_val = 0
        for [x, y] in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            y = y.to(device)
            predictions = model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss.backward()
            optimizer.step()
            loss_tr += loss.item() * len(x)
        loss_tr /= len(train_loader.dataset)
        for [x, y] in val_loader:
            x = x.to(device)
            y = y.to(device)
            predictions = model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss_val += loss.item() * len(x)
        loss_val /= len(val_loader.dataset)
        scheduler.step(loss_val)
        print(f"Epoch {epoch+1}: train loss: {loss_tr}, val loss: {loss_val}")
        if(optimizer.param_groups[0]['lr'] < 1e-6):
            print(f"Early stop at epoch {epoch+1}")
            break

    def make_features(x):
        """
        This function extracts features from the training and test data, used in the actual pipeline 
        after the pretraining.

        input: x: np.ndarray, the features of the training or test set

        output: features: np.ndarray, the features extracted from the training or test set, propagated
        further in the pipeline
        """
        model.eval()
        # TODO: Implement the feature extraction, a part of a pretrained model used later in the pipeline.
        x = torch.tensor(x, dtype=torch.float)
        x = x.to(device)
        x = model.make_feature(x)
        return x

    return make_features

In [6]:
def make_pretraining_class(feature_extractors):
    """
    The wrapper function which makes pretraining API compatible with sklearn pipeline
    
    input: feature_extractors: dict, a dictionary of feature extractors

    output: PretrainedFeatures: class, a class which implements sklearn API
    """

    class PretrainedFeatures(BaseEstimator, TransformerMixin):
        """
        The wrapper class for Pretraining pipeline.
        """
        def __init__(self, *, feature_extractor=None, mode=None):
            self.feature_extractor = feature_extractor
            self.mode = mode

        def fit(self, X=None, y=None):
            return self

        def transform(self, X):
            assert self.feature_extractor is not None
            X_new = feature_extractors[self.feature_extractor](X)
            return X_new
        
    return PretrainedFeatures


In [7]:
def get_regression_model(X, y):

    class Model(nn.Module):
        """
        The model class, which defines our feature extractor used in pretraining.
        """
        def __init__(self):
            """
            The constructor of the model.
            """
            super().__init__()
            # TODO: Define the architecture of the model. It should be able to be trained on pretraing data 
            # and then used to extract features from the training and test data.
            self.fc3 = nn.Linear(64, 1)

            # nn.init.kaiming_uniform_(self.fc3.weight, nonlinearity='relu')
            nn.init.xavier_normal_(self.fc3.weight)

        def forward(self, x):
            """
            The forward pass of the model.

            input: x: torch.Tensor, the input to the model

            output: x: torch.Tensor, the output of the model
            """
            # TODO: Implement the forward pass of the model, in accordance with the architecture 
            # defined in the constructor.
            x = self.fc3(x)
            return x
    
    """
    This function returns the regression model used in the pipeline.

    input: None

    output: model: sklearn compatible model, the regression model
    """
    # TODO: Implement the regression model. It should be able to be trained on the features extracted
    # by the feature extractor.
    model = Model()
    model.to(device)
    model.train()

    # x = torch.tensor(X, dtype=torch.float)
    x = X.clone().detach()
    x = x.to(device)
    y = torch.tensor(y, dtype=torch.float)
    y = y.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = 1000
    for epoch in tqdm(range(epochs)):
        optimizer.zero_grad()
        predictions = model(x).squeeze(-1)
        loss = criterion(predictions, y)
        loss.backward()
        optimizer.step()
        scheduler.step(loss)
        if(epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}: train loss: {loss}")
        if(optimizer.param_groups[0]['lr'] < 1e-7):
            print(f"Early stop at epoch {epoch+1}, loss: {loss}")
            break

    return model

In [8]:
result_file = "results.csv"

In [9]:
# Load data
x_pretrain, y_pretrain, x_train, y_train, x_test = load_data()
print("Data loaded!")
# Utilize pretraining data by creating feature extractor which extracts lumo energy 
# features from available initial features
feature_extractor =  make_feature_extractor(x_pretrain, y_pretrain)
PretrainedFeatureClass = make_pretraining_class({"pretrain": feature_extractor})
pretrainedfeatures = PretrainedFeatureClass(feature_extractor="pretrain")

x_train_featured = pretrainedfeatures.transform(x_train)
x_test_featured = pretrainedfeatures.transform(x_test.to_numpy())
# regression model
regression_model = get_regression_model(x_train_featured, y_train)

y_pred = np.zeros(x_test.shape[0])
# TODO: Implement the pipeline. It should contain feature extraction and regression. You can optionally
# use other sklearn tools, such as StandardScaler, FunctionTransformer, etc.
y_pred = regression_model(x_test_featured).squeeze(-1).detach().cpu().numpy()

assert y_pred.shape == (x_test.shape[0],)
y_pred = pd.DataFrame({"y": y_pred}, index=x_test.index)
y_pred.to_csv(result_file, index_label="Id")
print("Predictions saved, all done!")

c:\Users\linan\.conda\envs\DL\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Data loaded!


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1: train loss: 0.22172975515072443, val loss: 0.028617265075445174
Epoch 2: train loss: 0.016385001859494618, val loss: 0.017981045350432396
Epoch 3: train loss: 0.011280111132835856, val loss: 0.019214819803833962
Epoch 4: train loss: 0.01313225786600794, val loss: 0.021417522490024567
Epoch 5: train loss: 0.013379970808190351, val loss: 0.01715828260779381
Epoch 6: train loss: 0.01229264594796969, val loss: 0.016820656806230544
Epoch 7: train loss: 0.011458797817783696, val loss: 0.016914474219083785
Epoch 8: train loss: 0.011887369862472524, val loss: 0.011819722503423691
Epoch 9: train loss: 0.010222963360043205, val loss: 0.013717557489871978
Epoch 10: train loss: 0.010240353781805963, val loss: 0.013595572598278522
Epoch 11: train loss: 0.010367008017186, val loss: 0.01895168496668339
Epoch 12: train loss: 0.013771282754990519, val loss: 0.020211945101618767
Epoch 13: train loss: 0.011134911150348429, val loss: 0.016097051672637464
Epoch 00014: reducing learning rate of gro

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 10: train loss: 2.121460199356079
Epoch 20: train loss: 1.4736509323120117
Epoch 30: train loss: 1.0016741752624512
Epoch 40: train loss: 0.6867256164550781
Epoch 50: train loss: 0.4960576593875885
Epoch 60: train loss: 0.3925231993198395
Epoch 70: train loss: 0.342799574136734
Epoch 80: train loss: 0.32201364636421204
Epoch 90: train loss: 0.3145182430744171
Epoch 100: train loss: 0.3120565414428711
Epoch 110: train loss: 0.31104913353919983
Epoch 120: train loss: 0.310285747051239
Epoch 130: train loss: 0.309492826461792
Epoch 140: train loss: 0.3086583912372589
Epoch 150: train loss: 0.3078019618988037
Epoch 160: train loss: 0.3069305121898651
Epoch 170: train loss: 0.3060436248779297
Epoch 180: train loss: 0.3051401376724243
Epoch 190: train loss: 0.3042202293872833
Epoch 200: train loss: 0.3032849431037903
Epoch 210: train loss: 0.30233535170555115
Epoch 220: train loss: 0.30137214064598083
Epoch 230: train loss: 0.3003963828086853
Epoch 240: train loss: 0.2994089126586914
E